# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [7]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [1]:
import requests
response = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

Use Beautiful Soup to parse this string into a tree called `soup`

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [3]:
len(soup.find_all("table"))

14

We can use attributes like `class=` and `style=` to narrow down the list.

In [4]:
len(soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  ))

3

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [5]:
table = soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  )[0]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [8]:
df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", 
                          attrs = {'class': 'wikitable sortable', "style": "text-align:center"})[0]
df_cities2

,2022 rank,City,State[c],2022 estimate,2020 census,Change,2020 land area,2020 land area.1,2020 population density,2020 population density.1,Location
0,1,New York[d],New York,8335897,8804190,−5.32%,300.5 sq mi,778.3 km2,"29,298/sq mi","11,312/km2",40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W
1,2,Los Angeles,California,3822238,3898747,−1.96%,469.5 sq mi,"1,216.0 km2","8,304/sq mi","3,206/km2",34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W
2,3,Chicago,Illinois,2665039,2746388,−2.96%,227.7 sq mi,589.7 km2,"12,061/sq mi","4,657/km2",41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W
3,4,Houston,Texas,2302878,2304580,−0.07%,640.4 sq mi,"1,658.6 km2","3,599/sq mi","1,390/km2",29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W
4,5,Phoenix,Arizona,1644409,1608139,+2.26%,518.0 sq mi,"1,341.6 km2","3,105/sq mi","1,199/km2",33°34′N 112°05′W﻿ / ﻿33.57°N 112.09°W
...,...,...,...,...,...,...,...,...,...,...,...
328,329,Albany,New York,100826,99224,+1.61%,21.4 sq mi,55.4 km2,"4,637/sq mi","1,790/km2",42°40′N 73°48′W﻿ / ﻿42.67°N 73.80°W
329,330,Hesperia,California,100744,99818,+0.93%,72.7 sq mi,188.3 km2,"1,373/sq mi",530/km2,34°24′N 117°19′W﻿ / ﻿34.40°N 117.32°W
330,331,New Bedford,Massachusetts,100682,101079,−0.39%,20.0 sq mi,51.8 km2,"5,054/sq mi","1,951/km2",41°40′N 70°56′W﻿ / ﻿41.66°N 70.94°W
331,332,Davenport,Iowa,100486,101724,−1.22%,63.8 sq mi,165.2 km2,"1,594/sq mi",615/km2,41°34′N 90°36′W﻿ / ﻿41.56°N 90.60°W


In [23]:
rows = []

for city in table.find_all("tr")[1:]:
    cells = city.find_all("td")
    
    city_tag = cells[0].find("tr") or cells[0]
    city = city_tag.text

    state_tag = cells[1].find("td") or cells[1]
    state = state_tag.text

    population_tag = cells[2].find("td") or cells[2]
    population = population_tag.text
    
    land_area_tag = cells[5].find("td") or cells[5]
    land_area = land_area_tag.text
   
    rows.append({
        "city": "city",
        "state": "state",
        "2022 estimate": "population",
        "2020 land area": "sq mi"
    })
  

In [24]:
pd.DataFrame(rows)

,city,state,2022 estimate,2020 land area
0,city,state,population,sq mi
1,city,state,population,sq mi
2,city,state,population,sq mi
3,city,state,population,sq mi
4,city,state,population,sq mi
...,...,...,...,...
328,city,state,population,sq mi
329,city,state,population,sq mi
330,city,state,population,sq mi
331,city,state,population,sq mi


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [ ]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [ ]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [ ]:
df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})[0]
df_cities2

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.